# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd # <импорт библиотеки pandas>

Прочитаем файл datasets/data.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('/datasets/data.csv') # <чтение файла с данными с сохранением в df>

Получение первых 10 строк таблицы.

In [3]:
df.head(10) # <получение первых 10 строк таблицы df>

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы df.

In [4]:
df.info() # <получение общей информации о данных в таблице df>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов с информацией о заемщиках. Из них 2 столбца типа float64, 5 столбцов типа int64 и 5 столбцов типа object.

Подробно разберём, какие в df столбцы и какую информацию они содержат:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

Количество значений в некоторых столбцах (days_employed, total_income) отличается от общего количества строк. Это говорит о том, что в данных есть пропущенные значения.

**Вывод**

Каждая строка таблицы содержит информацию о клиенте, который оформлял один из существующих в данном банке видов кредита. Как правило указанные параметры используются для оценки платежеспособности клиента. Проблема, которую нужно решать: пропуски в указанных выше столбцах, наличие и обработка дубликатов, цели кредита записаны очевидно в свободной форме. Для удобства анализа цели необходимо будет привести к единой лексической форме.

## Шаг 2. Предобработка данных

Определим и заполним пропущенные значения,заменим вещественный тип данных на целочисленный, удалим дубликаты, выделим леммы в значениях столбца с целями получения кредита, категоризируем данные.

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [5]:
print(df.isnull().sum()) # <суммарное количество пропусков, выявленных методом isnull() в таблице df>

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пустые значения в столбцах days_employed и total_income свидетельствуют о том, что доступна не вся информация. Причины могут быть разные. 
Данные в столбце days_employed могут принимать отрицательные значения. Предположим, что подсчет этого столбца был автоматизирован внутренней системой учета стажа клиентов банка (например, для удобства подсчета накопленных пенсионных баллов или для других целях, которые не пересекаются с целями текущего проекта). Для удобства заменим пропуски в этом столбце нулями.

In [6]:
#замена пропусков на нули
df['days_employed'] = df['days_employed'].fillna(value=0)
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Пропущенных значений в столбце total_income составляет около 10% от общего числа строк. Как правило, банки требуют подтверждения совокупного дохода клиентов справками с места работы.Что могло послужить причиной пропусков в таблице? Причин может быть много. Например, акции для привлечения новых категорий заемщиков, у которых в текущий момент нет источника дохода, но представляющие потенциальный интерес для банка (студенты или родители, находящиеся в декретном отпуске).
Этот параметр значителен для выполнения целей проекта, поэтому пропуски нельзя заменять нулями как в случае выше. Попробуем заменить их значением медианы дохода по каждой группе занятости.

In [7]:
#рассчет медианного дохода для каждой группы занятости
median_income = df.groupby('income_type')['total_income'].median()
print(median_income)
 
#заполнение пропусков в столбе совокупного дохода на значение медианного дохода каждой группы занятости
for inc_type in median_income.index:
    df.loc[df['income_type'] == inc_type,'total_income'] = df.loc[df['income_type'] == inc_type,'total_income'].fillna(median_income[inc_type])

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [8]:
# <проверка: вычисление суммарного количества пропусков, выявленных в таблице df>
print(df.isnull().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

При изучении данных мы обнаружили пропуски данных (значения типа NaN в столбцах days_employed и total_income). Для обработки пропусков мы выбрали метод заполнения медианой.
Теперь в таблице нет пропусков и можно переходить к следующему этапу предобработки.

### Замена типа данных

В начале проекта методом info было установлено, что данные содержат 2 столбца с вещественным типом данных (float64) и по 5 столбцов - с целочисленным (int64) и нечисловыми типами данных (object). Для увеличения точности последующих вычислений и избежания неявных ошибок, необходимо преобразовать тип данных float в int64. 

Для начала разберемся со столбцом days_employed - общий трудовой стаж в днях. 
Для перевода данных столбца в целые числа воспользуемся методом astype(), таким образом учитывая только дни, отработанные полностью.


In [9]:
#перевод данных столбца days_employed в целые числа
df['days_employed'] = df['days_employed'].astype('int')

Также поступим со столбцом total_income, приведя ежемесячный доход клиентов банка в целое число. Отброшенная информация о неучтенных копейках не повлияет существенным образом на выводы о платежеспособности заемщиков.

In [10]:
#перевод данных столбца total_income в целые числа
df['total_income'] = df['total_income'].astype('int')

Проверим, остались ли в таблице df столбцы с типом данных  float после проведенных операций.

In [11]:
# проверка информации о данных в таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Были обработаны данные в таблице df, после чего не осталось столбцов с вещественными типами данных. Это поможет избежать ошибок в вычислениях, так как теперь все числовые данные унифицированы по типу.

### Обработка дубликатов

Перед тем, как обработать дублирующие строки, взглянем на общий вид первых строк таблицы

In [12]:
#получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Как видно из таблицы, нечисловые столбцы содержат разноформатную дублирующую информацию. Приведем эти столбцы в единый вид по нижнему регистру.

In [13]:
#перевод данных нечисловых столбцов к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

Проверим наличие дубликатов методом duplicated(). Затем дубликаты удаляем и проверяем таблицу.

In [14]:
#получение суммарного количества дубликатов в таблице df
df.duplicated().sum() 

71

In [15]:
#удаление всех дубликатов из таблицы df
df = df.drop_duplicates().reset_index(drop=True) 

In [16]:
#проверка на отсутствие дубликатов
print(df.duplicated().sum())

0


**Вывод**

Таблицу избавили от дубликатов специальным методом drop_duplicates.Появились они вероятнее всего вследствие технического сбоя в записи данных.

### Лемматизация

Определим количество категорий займа.

In [17]:
#импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()

# определение количества уникальных значений
print('Всего уникальных значений в столбце purpose:', len(list(df['purpose'].unique())), '\n')

# вывод всех значений столбца
print(df['purpose'].value_counts())

Всего уникальных значений в столбце purpose: 38 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля  

Как видно из выпавшего списка, всего уникальных значений целей кредита - 38. Изучив полученные данные, выделим следующие основные цели займа:
1) Покупка недвижимости (леммы "жильё", "недвижимость");
2) Покупка автомобиля (лемма "автомобиль");
3) Получение образования (лемма "образование");
4) Проведение свадьбы (лемма "свадьба");
5) Стройтельство недвижимости (лемма "строительство");
6) Ремонт недвижимости (лемма "ремонт").

Для того, чтобы привести все имеющиеся значения столбца purpose к списку с лематизированными целями, необходимо сделать следующее:

Создать новый датафрейм, в который поместить уникальные значения столбца purpose;
Написать функцию, которая принимает на вход строку, лемматизирует её и проверяет вхождение определённых для каждой категории лемм в полученный список, после чего возвращает одну из категорий;
Создать в новом датафрейме новый столбец, применив данную функцию к существующему столбцу. Таким образом каждому из большого количество уникальных значений столбца purpose будет сопоставлена одна из 6 определённых нами целей займа;
С помощью функции merge объединить основной и созданный нами новый датафреймы.

Также с помощью функции добавим к датафрейму столбец с цифровыми индексами для каждого типа займа. Для этого сначала составим словарь вида {'цель_займа' : 'индекс'}, а после в теле функции будем возвращать значения нового столбца по ключу.

In [18]:
# создаём новый датафрейм

p_l = pd.DataFrame({'purpose' : df['purpose'].unique()})

# функция для распределения лематизированных целей на шесть категорий
def lemm_cat(row):
    s = m.lemmatize(row['purpose'])
    if ('жилье' in s) or ('недвижимость' in s):
        if 'ремонт' in s:
            return 'Займ на ремонт недвижимости'
        elif 'строительство' in s:
            return 'Займ на строительство недвижимости'
        else:
            return 'Займ на покупку недвижимости'
    if 'автомобиль' in s:
        return 'Займ на покупку автомобиля'
    if 'образование' in s:
        return 'Займ на получение образования'
    if 'свадьба' in s:
        return 'Займ на проведение свадьбы'

# применим функцию для нового датафрейма и создадим новый столбец purpose_cat
p_l['purpose_cat'] = p_l.apply(lemm_cat, axis=1)

# создание словаря для последующего добавления столбца с цифровыми индексами для каждого типа займа
purpose_cat_ids = {'Займ на покупку недвижимости' : 0,
                  'Займ на покупку автомобиля' : 1,
                  'Займ на получение образования' : 2,
                  'Займ на проведение свадьбы' : 3,
                  'Займ на строительство недвижимости' : 4,
                  'Займ на ремонт недвижимости' : 5}

# напишем функцию для присвоения цифрового индекса
def purpose_id(row):
    return purpose_cat_ids[row['purpose_cat']]

# добавим цифровой индекс для каждой цели займа
p_l['purpose_cat_id'] = p_l.apply(purpose_id, axis=1)

# объединяем датафреймы
data = df.merge(p_l, on = 'purpose')

# посмотрим на объединённые данные
display(data.sort_values(by = 'total_income', ascending = False).head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,purpose_cat_id
20704,0,-1477,44,высшее,0,женат / замужем,0,M,компаньон,0,2265604,ремонт жилью,Займ на ремонт недвижимости,5
8549,1,-2577,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,строительство недвижимости,Займ на строительство недвижимости,4
1295,1,-5248,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,дополнительное образование,Займ на получение образования,2
4812,0,-4719,61,среднее,1,не женат / не замужем,4,F,сотрудник,0,1715018,покупка жилья для семьи,Займ на покупку недвижимости,0
2183,0,-5734,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,сыграть свадьбу,Займ на проведение свадьбы,3
16852,0,-2285,43,среднее,1,женат / замужем,0,M,компаньон,0,1597613,операции с недвижимостью,Займ на покупку недвижимости,0
15122,1,-333,41,высшее,0,гражданский брак,1,M,компаньон,0,1551152,свадьба,Займ на проведение свадьбы,3
14369,1,-3173,41,высшее,0,не женат / не замужем,4,F,компаньон,0,1427934,автомобиль,Займ на покупку автомобиля,1
11313,1,-10207,64,высшее,0,в разводе,3,M,компаньон,0,1350245,жилье,Займ на покупку недвижимости,0
5789,1,-1851,36,высшее,0,гражданский брак,1,F,сотрудник,0,1286280,покупка коммерческой недвижимости,Займ на покупку недвижимости,0


**Вывод**

Вместо 38 изначальных целей займа, дублировавших друг друга по смыслу, у нас теперь 6 категорий, охватывающих все изначальные цели. Это в дальнейшем облегчит решение задач исследования, так как теперь группировать данные и отслеживать зависимости стало гораздо проще. 
Для этого мы:
Определили и посчитали все уникальные значения столбца purpose;
Выделили на основании полученных данных 6 основных категорий займа;
Присвоили этим категориям цифровые индексы;
С помощью функции сопоставили каждой из оригинальных целей одну из шести полученных категорий и соответствующий цифровой индекс;
Добавили полученные значения в основные данные.

### Категоризация данных

Для того, чтобы ответить на 3й вопрос в проекте ("Есть ли зависимость между уровнем дохода и возвратом кредита в срок?") необходимо эти уровни дохода определить, то есть категоризировать данные по столбцу total_income. Для этого сначала обозначим предполагаемые категории дохода и их цифровые индексы, а также границы каждой из категорий. Границы будем определять в процентном соотношении от размаха данных по столбцу total_income. Чтобы получить равнорелевантные категории, определим их границы через процентили. Предполагаемых категорий будет 4:

0 - Невысокий доход - [min, 25-й процентиль]
1 - Средний доход - [25-й процентиль, 50-й процентиль]
2 - Высокий доход - [50-й процентиль, 75-й процентиль]
3 - Сверхвысокий доход - [75-й процентиль, max]

Для категоризации необходимо будет:

Посчитать границы категорий
Составить словарь с определениями наших категорий (индекс, название)
Написать функцию, которая будет проверять вхождение аргумента в одну из категорий и возвращать её наименование
Применить эту функцию к столбцу total_income и получить новый столбец total_income_cat
Написать функцию, которая будет возвращать индекс категории по её имени из словаря
Применить эту функцию к столбцу total_income_cat и получить новый столбец total_income_cat_id

In [23]:
# смотрим на данные и записываем результат работы describe() в отдельный список, чтобы не пересчитывать значения каждый раз
total_income_decription = list(df.total_income.describe())
display(df.total_income.describe())

# минимум, максимум
min = total_income_decription[3]
max = total_income_decription[7]

# правая граница первой категории
b_25 = total_income_decription[4]

# правая граница второй категории
b_50 = total_income_decription[5]

# правая граница третьей категории
b_75 = total_income_decription[6]

# создание словаря

total_income_cat_ids = {'Невысокий доход' : 0,
                       'Средний доход' : 1,
                       'Высокий доход' : 2,
                       'Сверхвысокий доход' : 3}

# функция для присвоения категории
def income_cat(row):
    s = row['total_income']
    if s <= b_25:
        return 'Невысокий доход'
    if s <= b_50:
        return 'Средний доход'
    if s <= b_75:
        return 'Высокий доход'
    if s <= max:
        return 'Сверхвысокий доход'

# столбец с категориями дохода
df['total_income_cat'] = df.apply(income_cat, axis=1)

# функция для получения цифрового индекса
def income_cat_id(row):
    return total_income_cat_ids[row['total_income_cat']]

# выведем что получилось

display(df.head(10))    

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_cat
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Сверхвысокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Высокий доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Сверхвысокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Высокий доход
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,Сверхвысокий доход
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,Сверхвысокий доход
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,Средний доход
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,Невысокий доход
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,Высокий доход


**Вывод**

Чтобы ответить на один из вопросов проекта, данные были категоризованы по уровню дохода. Для этого было сделано следующее:

Определено количество категорий, их границы, сопоставлены им цифровые индексы
Написана функция, которая присваивает категорию дохода в зависимости от его значения
Составлен словарь с названиями категорий и их цифровыми индексами
Написана функция, возвращающая цифровой индекс категории по её названию
Как результат, получили новые столбцы с категорией дохода и цифровым индексом данной категории.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
#выведем данные по количеству детей в семьях заемщиков
print(df['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Вероятнее всего, значение -1 в в столбце children - это техническая ошибка, можно просто убрать знак - с помощью функции abs.

In [25]:
df['children'] = df['children'].apply(abs)

In [26]:
df.groupby('children')['debt'].agg(['count', 'mean'])
report1 = df.pivot_table(index = 'children', columns = 'debt', values = 'dob_years', aggfunc = 'count').fillna(0)
report1.columns = ['no_debt','debt']
report1['percent'] = report1['debt'] / (report1['debt'] + report1['no_debt'])
display(report1.style.format({'percent':'{:.2%}', 'no_debt':'{:.0f}'}))

,no_debt,debt,percent
children,,,
0,13028,1063,7.54%
1,4410,445,9.17%
2,1858,194,9.45%
3,303,27,8.18%
4,37,4,9.76%
5,9,0,0.00%
20,68,8,10.53%


**Вывод**

Исходя из полученных данных можно сделать вывод, что зависимость между наличием детей и возвратом кредита в срок существует. Заёмщики без детей имеют самый низкий процент должников (не считая аномальной группы заёмщиков с 5-ю детьми, где должников нет вовсе). С увеличением числа детей процент должников в группе начинает возрастать (опять же за исключением групп с 3-мя и 5-ю детьми). В целом это логично - чем больше детей, тем выше финансовая нагрузка и выше риск невозврата займа в срок.

Процент по категориям, в которых трое и более детей, может быть недостоверным, так как выборки довольно малы и потому нерепрезентативны.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные по столбцу family_status и применим к столбцу debt функции sum и count для получения числа должников и общего числа записей в группе соответственно. Эти данные запишем в новый датафрейм.
Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе.

In [27]:
# создаём новый датафрейм с группировкой
family_data = df.groupby('family_status').agg({'debt' : ['sum', 'count']})

# добавляем столбец с процентом должников в каждой группе
family_data['debt_ratio'] = round(family_data['debt']['sum'] / family_data['debt']['count'] * 100,2)

# выводим полученную таблицу
display(family_data.sort_values(by = 'debt_ratio', ascending = False))

debt        debt_ratio
                       sum  count           
family_status                               
не женат / не замужем  274   2810       9.75
гражданский брак       388   4151       9.35
женат / замужем        931  12339       7.55
в разводе               85   1195       7.11
вдовец / вдова          63    959       6.57

**Вывод**

На основе полученных данных можно сделать вывод о том, что зависимость между семейным положением и возвратом кредита в срок существует. Самые высокие доли должников среди незамужних/неженатых и тех, кто предпочел не регистрировать брак официально. Должников среди состоящих (и состоявших) в официально зарегистрированных отношениях - существенно меньше. Такой результат связан с тем, что решение бытовых вопросов вскладчину всегда выходит экономичнее, поэтому в большинстве семей ведется совместный бюджет. А это, в свою очередь, что позволяет оптимизировать денежные потоки на актуальные цели.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


Сгруппируем данные по столбцу total_income_cat и применим к столбцу debt функции sum и count для получения числа должников и общего числа записей в группе соответственно. Эти данные запишем в новый датафрейм
Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе

In [28]:
# создаём новый датафрейм с группировкой
income_data = df.groupby('total_income_cat').agg({'debt' : ['sum', 'count']})

# добавляем столбец с процентом должников к общему числу в группе
income_data['debt_ratio'] = round(income_data['debt']['sum'] / income_data['debt']['count'] * 100, 2)

# выводим полученную таблицу
display(income_data.sort_values(by = 'debt_ratio', ascending = False))


debt       debt_ratio
                    sum count           
total_income_cat                        
Средний доход       483  5479       8.82
Высокий доход       448  5247       8.54
Невысокий доход     427  5364       7.96
Сверхвысокий доход  383  5364       7.14

**Вывод**

Прослеживается определенная логика между уровнем дохода и возвратом кредита в срок. 
Меньше всего должников в категории со сверхвысоким доходом (7,14 %). Наличие сверхвысокого дохода позволяет поддерживать финансовую дисциплину.

Немного должников (7,96 %) и среди обладателей невысокого дохода, что может показаться контринтуитивным сначала.Однако, именно ограниченность бюджета, как правило, заставляет людей быть финансово дисциплинированными: выплачивать кредиты вовремя, избегать начисления штрафов и пеней, а также поддерживать хорошую кредитную историю.

Остальные категории дохода (средний и высокий) имеют примерно равный процент должников (8,82 % и 8,54 % соответственно). Возможно, данным категориям уже не так страшны штрафы в силу более высокого дохода.

Отвечая на вопрос, можно сказать, что зависимость между уровнем дохода и возвратом кредита в срок существует.

- Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу, сгруппированную по целям кредита. В качестве значений, по которым мы хотим увидеть сводку, возмьём значения столбца debt. 
Добавим в полученный датафрейм ещё один столбец, в который запишем отношение числа должников к общему числу в группе и умножим на 100. Результат округлим до 2 чисел после запятой. Таким образом получим процент должников в каждой группе.

In [29]:
# создадим новый датафрейм, куда сохраним сводную таблицу по должникам
purpose_data = data.pivot_table(index = 'purpose_cat', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
purpose_data['mean'] = round(purpose_data['mean'] * 100, 2)

# выводим полученную таблицу
display(purpose_data)

,sum,count,mean
,debt,debt,debt
purpose_cat,,,
Займ на покупку автомобиля,403,4306,9.36
Займ на покупку недвижимости,603,8326,7.24
Займ на получение образования,370,4013,9.22
Займ на проведение свадьбы,186,2324,8.00
Займ на ремонт недвижимости,35,607,5.77
Займ на строительство недвижимости,144,1878,7.67


**Вывод**

Можно проследить зависимость между целью кредита и процентом невозврата в срок:

Самый низкий процент просрочек (5,77 %) - среди займов на ремонт, ввиду наличия сметы и сравнительно небольшой суммы кредита.
Покупка и строительство недвижимости (7,24 % и 7,67 %) - серьёзные шаги, которые также предполагают серьёзный предварительный расчёт.
Свадьба (8 %) - кредит на свадьбу берется из установки "один раз в жизни свадьба", закатывая порой мероприятия не по средствам.
Займ на получение образования (9,22 %) - повышенный процент просрочек предположительно связан с тем, недавние выпускники не сразу находят высокооплачиваемую работу и им необходимо какое-то время для профессионального становления, и соответственно, повышения доходов.
Автомобиль (9,36 %) - лидер по числу просрочек, так как многие берут автомобили в кредит не по карману, откуда и максимальный процент просрочек.

## Шаг 4. Общий вывод

Основная задача этого проекта - выделить ненадежные группы заемщиков.
В результате выявлена зависимость между наличием детей и возвратом кредита в срок: наиболее рискованно выдавать кредиты семьям с 1-2 детьми (9-10% процентов таких клиентов задерживают выплаты по кредитам). Процент по категориям, в которых трое и более детей, может быть недостоверным, так как выборки довольно малы и потому нерепрезентативны
Также отслеживается зависимость между семейным положением и возвратом кредита в срок - холостые и люди в гражданском браке более склонны к задолженностям по кредиту.
Выявлена взаимосвязь между уровнем дохода и возвратом кредита в срок - люди со средним доходом - самые рискованные плательщики (их 8.82%). Наименее рискованные плательщики - люди с высоким доходом.
В результате исследования зависимости цели кредита и его возврата в срок, можно сделать вывод, что наиболее рискованные цели кредита - это категории, связанные с автомобилями и образованием. Они наиболее подвержены невозврату в срок.
По полученным данным можно сделать вывод, что при любых условиях не менее 90% клиентов возвращают кредит в срок. Поэтому кредитование является довольно популярным бизнесом.
Хорошие клиенты, которые возвращают кредит в срок:
клиенты, не имеющие детей. 92,5% процентов таких клиентов возвращают кредит в срок;
92.5% и 93.5% (соответственно) клиентов, находящихся в семейном положении "вдовец/вдова" и "женат/замужем" своевременно возвращают кредит;
наименее рискованные плательщики - люди со сверхвысоким уровнем дохода - 93% таких клиентов возвращают кредиты в срок;
наименее рискованные цели взятия кредита - это ремонт и покупка недвижимости.